**importation du bébliothéque** 

In [82]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from textblob import TextBlob
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation # LDA
from sklearn.model_selection import train_test_split , GridSearchCV 
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler



**Chargement dataset "dreaddit_StressAnalysis.csv"**

In [63]:
path='dreaddit_StressAnalysis.csv'
df=pd.read_csv(path)
df.head()

,id,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,896,relationships,7nu7as,"[50, 55]","Its like that, if you want or not.“ ME: I have...",0,0.8,1514980773,22,-1.238793,...,1.0000,1.2000,1.0,1.65864,1.32245,1.80264,0.63,62,-0.148707,0.000000
1,19059,anxiety,680i6d,"(5, 10)",I man the front desk and my title is HR Custom...,0,1.0,1493348050,5,7.684583,...,1.4000,1.1250,1.0,1.69133,1.69180,1.97249,1.00,2,7.398222,-0.065909
2,7977,ptsd,8eeu1t,"(5, 10)",We'd be saving so much money with this new hou...,1,1.0,1524516630,10,2.360408,...,1.1429,1.0000,1.0,1.70974,1.52985,1.86108,1.00,8,3.149288,-0.036818
3,1214,ptsd,8d28vu,"[2, 7]","My ex used to shoot back with ""Do you want me ...",1,0.5,1524018289,5,5.997000,...,1.0000,1.3000,1.0,1.72615,1.52000,1.84909,1.00,7,6.606000,-0.066667
4,1965,relationships,7r1e85,"[23, 28]",I haven’t said anything to him yet because I’m...,0,0.8,1516200171,138,4.649418,...,1.1250,1.1429,1.0,1.75642,1.43582,1.91725,0.84,70,4.801869,0.141667


In [64]:
print("les dimension de dataset",df.shape)
print("les colonnes de dataset",df.columns.tolist())

les dimension de dataset (715, 116)
les colonnes de dataset ['id', 'subreddit', 'post_id', 'sentence_range', 'text', 'label', 'confidence', 'social_timestamp', 'social_karma', 'syntax_ari', 'lex_liwc_WC', 'lex_liwc_Analytic', 'lex_liwc_Clout', 'lex_liwc_Authentic', 'lex_liwc_Tone', 'lex_liwc_WPS', 'lex_liwc_Sixltr', 'lex_liwc_Dic', 'lex_liwc_function', 'lex_liwc_pronoun', 'lex_liwc_ppron', 'lex_liwc_i', 'lex_liwc_we', 'lex_liwc_you', 'lex_liwc_shehe', 'lex_liwc_they', 'lex_liwc_ipron', 'lex_liwc_article', 'lex_liwc_prep', 'lex_liwc_auxverb', 'lex_liwc_adverb', 'lex_liwc_conj', 'lex_liwc_negate', 'lex_liwc_verb', 'lex_liwc_adj', 'lex_liwc_compare', 'lex_liwc_interrog', 'lex_liwc_number', 'lex_liwc_quant', 'lex_liwc_affect', 'lex_liwc_posemo', 'lex_liwc_negemo', 'lex_liwc_anx', 'lex_liwc_anger', 'lex_liwc_sad', 'lex_liwc_social', 'lex_liwc_family', 'lex_liwc_friend', 'lex_liwc_female', 'lex_liwc_male', 'lex_liwc_cogproc', 'lex_liwc_insight', 'lex_liwc_cause', 'lex_liwc_discrep', 'lex_liw

In [65]:
print ("les valeurs manquantes par colones\n", df.isnull().sum())

les valeurs manquantes par colones
 id                          0
subreddit                   0
post_id                     0
sentence_range              0
text                        0
                           ..
lex_dal_avg_pleasantness    0
social_upvote_ratio         0
social_num_comments         0
syntax_fk_grade             0
sentiment                   0
Length: 116, dtype: int64


In [66]:
print("suppression des doublons : ", df.duplicated().sum())

suppression des doublons :  0


colone to keep :
**colones de base**
2. text
3. label : stressé ou pas 
4. social_timeslamp : analyse temporelle

**colones santé et emotions**
1. lex_dal_avg_activation : Clé pour différencier fatigue et panique
2. lex_liwc_anx, lex_liwc_anger, lex_liwc_sad : Scores spécifiques pour l'anxiété, la colère et la tristesse
3. lex_liwc_health : Indique si le sujet parle de problèmes de santé physique.

**Colonnes Médias & Social**
1. subreddit : savoir d'où vient le stress

**Colonnes Nature du Stress**
1. lex_liwc_work(élevé) : détecter le stress Organisationnel.
2. lex_liwc_family ou lex_liwc_friend (élevé): détecter le stress Émotionnel/Relationnel.
3. lex_liwc_i : Un usage élevé du "Je" est souvent lié à une forte subjectivité et une détresse personnelle.

**d'autre colones**
1. sentiment : un score de polarité 
Valeur < 0 : négatif (colère, tristesse, douleur).
Valeur = 0 :  neutre (informatif, factuel).
Valeur > 0 : positif (remerciements, espoir, soulagement).


In [67]:
cols_to_keep=['text', 'label', 'social_timestamp','subreddit','lex_dal_avg_activation','lex_liwc_anx', 'lex_liwc_anger', 'lex_liwc_sad', 
              'lex_liwc_health','lex_liwc_work','lex_liwc_family','lex_liwc_friend', 'lex_liwc_i', 'sentiment']

print("les colones utiles a conserver\n", cols_to_keep)

les colones utiles a conserver
 ['text', 'label', 'social_timestamp', 'subreddit', 'lex_dal_avg_activation', 'lex_liwc_anx', 'lex_liwc_anger', 'lex_liwc_sad', 'lex_liwc_health', 'lex_liwc_work', 'lex_liwc_family', 'lex_liwc_friend', 'lex_liwc_i', 'sentiment']


In [68]:
print("nouvelle data avec des données utils")
df1=df[cols_to_keep]
df1.head()

nouvelle data avec des données utils


,text,label,social_timestamp,subreddit,lex_dal_avg_activation,lex_liwc_anx,lex_liwc_anger,lex_liwc_sad,lex_liwc_health,lex_liwc_work,lex_liwc_family,lex_liwc_friend,lex_liwc_i,sentiment
0,"Its like that, if you want or not.“ ME: I have...",0,1514980773,relationships,1.65864,0.00,0.0,0.00,0.00,0.00,0.0,1.82,5.45,0.000000
1,I man the front desk and my title is HR Custom...,0,1493348050,anxiety,1.69133,0.00,0.0,0.00,0.00,22.22,0.0,0.00,6.94,-0.065909
2,We'd be saving so much money with this new hou...,1,1524516630,ptsd,1.70974,0.68,0.0,1.35,2.70,0.68,0.0,0.68,6.08,-0.036818
3,"My ex used to shoot back with ""Do you want me ...",1,1524018289,ptsd,1.72615,0.00,0.0,1.00,0.00,1.00,1.0,1.00,12.00,-0.066667
4,I haven’t said anything to him yet because I’m...,0,1516200171,relationships,1.75642,0.00,0.0,0.00,1.28,0.00,0.0,2.56,11.54,0.141667


In [69]:
print("les subreddits : ",pd.unique(df1['subreddit']))
print("\n les colouns du nouveau dataset : ",df1.columns.tolist())

les subreddits :  ['relationships' 'anxiety' 'ptsd' 'assistance' 'homeless' 'almosthomeless'
 'domesticviolence' 'survivorsofabuse' 'stress' 'food_pantry']

 les colouns du nouveau dataset :  ['text', 'label', 'social_timestamp', 'subreddit', 'lex_dal_avg_activation', 'lex_liwc_anx', 'lex_liwc_anger', 'lex_liwc_sad', 'lex_liwc_health', 'lex_liwc_work', 'lex_liwc_family', 'lex_liwc_friend', 'lex_liwc_i', 'sentiment']


**Pour la CRÉATION (Feature Extraction)**

In [70]:
print("Calcul de la subjectivité et polarité...")
# On calcule les scores de TextBlob sur la colonne 'text'
df1['polarity'] = df1['text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
df1['subjectivity'] = df1['text'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)


# Index de Vulnérabilité (Santé + Tristesse + Repli sur soi 'I')
df1['distress_index'] = df1['lex_liwc_health'] + df1['lex_liwc_sad'] + df1['lex_liwc_i']
# Index de Burnout (Travail + Anxiété + Colère)
df1['burnout_index'] = df1['lex_liwc_work'] + df1['lex_liwc_anx'] + df1['lex_liwc_anger']


print("Conversion des dates...")
# Conversion du timestamp UNIX en format Date lisible
df1['date'] = pd.to_datetime(df1['social_timestamp'], unit='s')

#simplification des labels du sectreur stressé : au lieu d'aprendre 10, en apprend 4
health_map={ 'ptsd': 'trauma' , 'domesticviolence': 'Trauma', 'survivorsofabuse': 'Trauma',
    'assistance': 'Précarité', 'homeless': 'Précarité', 'almosthomeless': 'Précarité', 'food_pantry': 'Précarité',
    'anxiety': 'Psychologique', 'stress': 'Psychologique',
    'relationships': 'Social', 'depression': 'Psychologique'}

df1['subreddit']=df1['subreddit'].map(health_map)
df1.head()

Calcul de la subjectivité et polarité...


C:\Users\Dell\AppData\Local\Temp\ipykernel_13232\3266732701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['polarity'] = df1['text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)


Conversion des dates...


C:\Users\Dell\AppData\Local\Temp\ipykernel_13232\3266732701.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['subjectivity'] = df1['text'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)
C:\Users\Dell\AppData\Local\Temp\ipykernel_13232\3266732701.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['distress_index'] = df1['lex_liwc_health'] + df1['lex_liwc_sad'] + df1['lex_liwc_i']
C:\Users\Dell\AppData\Local\Temp\ipykernel_13232\3266732701.py:10: SettingWithCopyWarning: 
A value is tryin

,text,label,social_timestamp,subreddit,lex_dal_avg_activation,lex_liwc_anx,lex_liwc_anger,lex_liwc_sad,lex_liwc_health,lex_liwc_work,lex_liwc_family,lex_liwc_friend,lex_liwc_i,sentiment,polarity,subjectivity,distress_index,burnout_index,date
0,"Its like that, if you want or not.“ ME: I have...",0,1514980773,Social,1.65864,0.00,0.0,0.00,0.00,0.00,0.0,1.82,5.45,0.000000,0.000000,0.000000,5.45,0.00,2018-01-03 11:59:33
1,I man the front desk and my title is HR Custom...,0,1493348050,Psychologique,1.69133,0.00,0.0,0.00,0.00,22.22,0.0,0.00,6.94,-0.065909,-0.065909,0.513636,6.94,22.22,2017-04-28 02:54:10
2,We'd be saving so much money with this new hou...,1,1524516630,trauma,1.70974,0.68,0.0,1.35,2.70,0.68,0.0,0.68,6.08,-0.036818,-0.036818,0.500303,10.13,1.36,2018-04-23 20:50:30
3,"My ex used to shoot back with ""Do you want me ...",1,1524018289,trauma,1.72615,0.00,0.0,1.00,0.00,1.00,1.0,1.00,12.00,-0.066667,-0.066667,0.550000,13.00,1.00,2018-04-18 02:24:49
4,I haven’t said anything to him yet because I’m...,0,1516200171,Social,1.75642,0.00,0.0,0.00,1.28,0.00,0.0,2.56,11.54,0.141667,0.141667,0.695370,12.82,0.00,2018-01-17 14:42:51


In [71]:
# savoir la correlation entre polarity , subjectivity et sentiment 
coorelation_matrix=df1[['polarity','subjectivity','sentiment']].corr()
print("Matrice de corrélation :\n",coorelation_matrix)

Matrice de corrélation :
               polarity  subjectivity  sentiment
polarity      1.000000     -0.083122   0.999739
subjectivity -0.083122      1.000000  -0.082985
sentiment     0.999739     -0.082985   1.000000


In [72]:
# On supprime les anciennes colonnes pour alléger le dataset
df2 = df1.drop(columns=['sentiment', 'social_timestamp'])

**Pré_traitement du dataset  avec NLP**

**phase 1 :**

a.	supprimer hashtage et url 

b.	tokenization : decoupage des phrase en mots individuals

c.	stops words (filtrage )

d.	lemmatisation : ramener les words à leurs racine verbe infinitif 

**phase 2 :** 

TF-IDF (Term Frequency-Inverse Document Frequency) : donner un score d’importance a chaque 


In [73]:
#phase 1

nlp= spacy.load('en_core_web_sm')

def preprocess_text(text):
    #covertir le txt en miniscule 
    text = text.lower()
    
    #supprimer les lien url 
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'@\w+|#\w+', '', text)
    
    #tokanisation 
    doc = nlp(text)
    
    #lemmatization
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)

df2['cleaned_text'] = df2['text'].apply(preprocess_text)
df2.head()   

,text,label,subreddit,lex_dal_avg_activation,lex_liwc_anx,lex_liwc_anger,lex_liwc_sad,lex_liwc_health,lex_liwc_work,lex_liwc_family,lex_liwc_friend,lex_liwc_i,polarity,subjectivity,distress_index,burnout_index,date,cleaned_text
0,"Its like that, if you want or not.“ ME: I have...",0,Social,1.65864,0.00,0.0,0.00,0.00,0.00,0.0,1.82,5.45,0.000000,0.000000,5.45,0.00,2018-01-03 11:59:33,like want problem take long ask friend help le...
1,I man the front desk and my title is HR Custom...,0,Psychologique,1.69133,0.00,0.0,0.00,0.00,22.22,0.0,0.00,6.94,-0.065909,0.513636,6.94,22.22,2017-04-28 02:54:10,man desk title hr customer service representat...
2,We'd be saving so much money with this new hou...,1,trauma,1.70974,0.68,0.0,1.35,2.70,0.68,0.0,0.68,6.08,-0.036818,0.500303,10.13,1.36,2018-04-23 20:50:30,save money new housr expensive city googling l...
3,"My ex used to shoot back with ""Do you want me ...",1,trauma,1.72615,0.00,0.0,1.00,0.00,1.00,1.0,1.00,12.00,-0.066667,0.550000,13.00,1.00,2018-04-18 02:24:49,ex shoot want time matter wonder block ask wed...
4,I haven’t said anything to him yet because I’m...,0,Social,1.75642,0.00,0.0,0.00,1.28,0.00,0.0,2.56,11.54,0.141667,0.695370,12.82,0.00,2018-01-17 14:42:51,say sure hear partner fluctuation feeling espe...


**Vectorisation**

Les algorithmes comme le SVM ou Naive Bayes sont des machines à calculer. Ils ne savent pas lire les lettres, ils ne comprennent que les nombres. La vectorisation est donc le traducteur qui transforme tes phrases nettoyées en une liste de coordonnées numériques.

In [74]:
# Phase 2 : La Vectorisation (TF-IDF)

"""TF (Term Frequency) : Est-ce que le mot "anxiety" revient souvent dans ce post ?
   IDF (Inverse Document Frequency) : Est-ce que le mot "anxiety" est rare et important,
   ou est-ce qu'il est présent partout ?"""
   
tfidf = TfidfVectorizer(max_features=3000)

#transformation du text en nombre
X_tfidf = tfidf.fit_transform(df2['cleaned_text']).toarray()
df2.head()

,text,label,subreddit,lex_dal_avg_activation,lex_liwc_anx,lex_liwc_anger,lex_liwc_sad,lex_liwc_health,lex_liwc_work,lex_liwc_family,lex_liwc_friend,lex_liwc_i,polarity,subjectivity,distress_index,burnout_index,date,cleaned_text
0,"Its like that, if you want or not.“ ME: I have...",0,Social,1.65864,0.00,0.0,0.00,0.00,0.00,0.0,1.82,5.45,0.000000,0.000000,5.45,0.00,2018-01-03 11:59:33,like want problem take long ask friend help le...
1,I man the front desk and my title is HR Custom...,0,Psychologique,1.69133,0.00,0.0,0.00,0.00,22.22,0.0,0.00,6.94,-0.065909,0.513636,6.94,22.22,2017-04-28 02:54:10,man desk title hr customer service representat...
2,We'd be saving so much money with this new hou...,1,trauma,1.70974,0.68,0.0,1.35,2.70,0.68,0.0,0.68,6.08,-0.036818,0.500303,10.13,1.36,2018-04-23 20:50:30,save money new housr expensive city googling l...
3,"My ex used to shoot back with ""Do you want me ...",1,trauma,1.72615,0.00,0.0,1.00,0.00,1.00,1.0,1.00,12.00,-0.066667,0.550000,13.00,1.00,2018-04-18 02:24:49,ex shoot want time matter wonder block ask wed...
4,I haven’t said anything to him yet because I’m...,0,Social,1.75642,0.00,0.0,0.00,1.28,0.00,0.0,2.56,11.54,0.141667,0.695370,12.82,0.00,2018-01-17 14:42:51,say sure hear partner fluctuation feeling espe...


**Sans fusion**

Tu passes directement X_tfidf dans ton split.

L'IA ne regarde que les mots. Elle devient un simple classifieur de texte.

Le problème : Tout le travail que tu as fait au début (calcul du distress_index, de la subjectivity, etc.) est perdu. Ton modèle ne les utilisera jamais pour prendre sa décision. C'est comme si un médecin écoutait ce que dit le patient mais refusait de regarder ses analyses de sang.

**avec fusion**

Tu passes X_final dans ton split.

L'IA regarde les mots + les indicateurs. Elle devient un outil de diagnostic multidimensionnel.

In [75]:
numeric_features = df2.select_dtypes(include=['float64', 'int64']).columns.tolist()
print("Les caractéristiques numériques :\n", numeric_features)

Les caractéristiques numériques :
 ['label', 'lex_dal_avg_activation', 'lex_liwc_anx', 'lex_liwc_anger', 'lex_liwc_sad', 'lex_liwc_health', 'lex_liwc_work', 'lex_liwc_family', 'lex_liwc_friend', 'lex_liwc_i', 'polarity', 'subjectivity', 'distress_index', 'burnout_index']


In [76]:
# 1. On prépare les scores numériques
numeric_cols = [col for col in numeric_features if col != 'label']

X_numeric_values = df2[numeric_cols].values

# 3. La fusion (Le Grand Pont)
# On colle les 3000 colonnes du texte et les 15 colonnes de scores
X_final = np.hstack([X_tfidf, X_numeric_values])

print(f"Fusion terminée ! Taille de la matrice finale : {X_final.shape}")


Fusion terminée ! Taille de la matrice finale : (715, 3013)


**Application de LDA**

un extracteur de sens.

In [79]:
#jai deja 4 surbeddits : Trauma , Précarité , Psychologique , Social
# je vais appliquer LDA pour trouver des topics dans chaque surbredit 

LDA = LatentDirichletAllocation(n_components=6, random_state=42)
lda_values=LDA.fit_transform(X_tfidf)

# 4. Ajout du thème dominant à ton DataFrame pour analyse
df2['lda_topic'] = lda_values.argmax(axis=1)
print(f"Dimensions de dataset   : {df2.shape}")

Dimensions de dataset   : (715, 19)


In [80]:
# Rappel : X_tfidf (3000) + X_numeric (15) + lda_values (4)
X_final_complete = np.hstack([X_tfidf, X_numeric_values, lda_values])

print(f"Nouvelle dimension de X_final : {X_final_complete.shape}")

Nouvelle dimension de X_final : (715, 3019)


**Modeling**

Data preperation + SVM

In [81]:
y= df2['label']

X_train, X_test, y_train, y_test = train_test_split(X_final_complete, y, test_size=0.2, random_state=42, stratify=y)

#grid search pour trouver les meilleurs hyperparametres pour SVM
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}

grid= GridSearchCV(
    SVC(),
    param_grid,
    refit=True,  # retrain on the whole dataset after finding the best params automatically
    verbose=2    # show progress logs in real time
)

print("Recherche des meilleurs paramètres en cours...")
grid.fit(X_train, y_train)

# 4. Résultats
print(f"Meilleur score trouvé : {grid.best_score_:.4f}")
print(f"Meilleur paramètre C : {grid.best_params_['C']}")

Recherche des meilleurs paramètres en cours...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END ...............................C=0.1, kernel=linear; total time=   0.3s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.3s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.5s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.2s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.4s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.4s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.5s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.3s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.3s
[CV] END ..................................C=0.1, kernel=rbf; total time=   0.3s
[CV] END .................................C=1, kernel=linear; total time=   0.3s
[C

K-means

In [84]:
indicateurs = [
    'distress_index', 'burnout_index', 'polarity', 'subjectivity',
    'lex_liwc_anx', 'lex_liwc_anger', 'lex_liwc_sad', 
    'lex_liwc_work', 'lex_liwc_family', 'lex_liwc_health']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df2[indicateurs])

kmeans= KMeans(n_clusters=6, random_state=42)
df2['cluster_id'] = kmeans.fit_predict(X_scaled)

print("Clusters ajoutés au DataFrame.", df2.shape)
df2.head()

Clusters ajoutés au DataFrame. (715, 20)


,text,label,subreddit,lex_dal_avg_activation,lex_liwc_anx,lex_liwc_anger,lex_liwc_sad,lex_liwc_health,lex_liwc_work,lex_liwc_family,lex_liwc_friend,lex_liwc_i,polarity,subjectivity,distress_index,burnout_index,date,cleaned_text,lda_topic,cluster_id
0,"Its like that, if you want or not.“ ME: I have...",0,Social,1.65864,0.00,0.0,0.00,0.00,0.00,0.0,1.82,5.45,0.000000,0.000000,5.45,0.00,2018-01-03 11:59:33,like want problem take long ask friend help le...,4,0
1,I man the front desk and my title is HR Custom...,0,Psychologique,1.69133,0.00,0.0,0.00,0.00,22.22,0.0,0.00,6.94,-0.065909,0.513636,6.94,22.22,2017-04-28 02:54:10,man desk title hr customer service representat...,0,2
2,We'd be saving so much money with this new hou...,1,trauma,1.70974,0.68,0.0,1.35,2.70,0.68,0.0,0.68,6.08,-0.036818,0.500303,10.13,1.36,2018-04-23 20:50:30,save money new housr expensive city googling l...,0,0
3,"My ex used to shoot back with ""Do you want me ...",1,trauma,1.72615,0.00,0.0,1.00,0.00,1.00,1.0,1.00,12.00,-0.066667,0.550000,13.00,1.00,2018-04-18 02:24:49,ex shoot want time matter wonder block ask wed...,1,0
4,I haven’t said anything to him yet because I’m...,0,Social,1.75642,0.00,0.0,0.00,1.28,0.00,0.0,2.56,11.54,0.141667,0.695370,12.82,0.00,2018-01-17 14:42:51,say sure hear partner fluctuation feeling espe...,4,0


**Sauvgarde d'un seul coup pkt**

In [91]:
import pickle

model_pack= {
    'tfidf': tfidf,               # Ton TfidfVectorizer
    'svm_model': grid.best_estimator_, # Ton meilleur modèle SVM (issu du GridSearch)
    'scaler': scaler,             # Ton StandardScaler
    'kmeans_model': kmeans,       # Ton modèle K-Means à 6 clusters
    'features_list': indicateurs # La liste des colonnes utilisées pour K-Means
}

# Sauvegarde dans un fichier binaire
with open('santemind_model.pkl', 'wb') as file:
    pickle.dump(model_pack, file)

print("✅ Succès : Tous les modèles et outils ont été sauvegardés !")


✅ Succès : Tous les modèles et outils ont été sauvegardés !
